### crawl all 980 popular science books on Douban

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen

import pandas as pd

import re

In [3]:
class base():
    
    def __init__(self,url):
        self.url = url
    
    def all_urls(self):
        all_urls_list = []
        i = 20
        while i <= 980:
            current_url = self.url + str(i) + '&type=T'
            all_urls_list.append(current_url)
            i += 20
        return all_urls_list

In [5]:
#get the category of popular science book 
url = 'https://book.douban.com/tag/科普?start='
urls = base(url).all_urls()

In [9]:
#get a list of 980 popular science books
url_list = []
for url in urls:   
    response = requests.get(url)
    contents = response.text
    soup = BeautifulSoup(contents)
    links = soup.find_all(title = True, onclick = True) 
    for item in links:
        url = item.get('href')
        url_list.append(url)

/Users/wanghanxuan/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/wanghanxuan/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [99]:
#remove html tage
dr = re.compile(r'<[^>]+>',re.S)

In [367]:
#get the detail of each book
def get_book_info(url):
    response = requests.get(url)
    contents = response.text
    soup = BeautifulSoup(contents)
    
    #title
    raw_title = soup.find_all('span', property = 'v:itemreviewed')
    title = dr.sub('',str(raw_title))
    title = title[1:-1]
    
    #author
    raw_basic_info = soup.find_all('div', id = 'info')
    raw_author = re.findall(r"href=(.+?)a>",str(raw_basic_info))

    if raw_author != []:
        raw_author = raw_author[0]
        author = re.findall(r">(.+?)</", raw_author)[0]
    else:
        raw_author = re.findall(r"(.+?)</a>",str(raw_basic_info))
        if str(raw_author) != '[]':
            author = re.sub(' ','',raw_author[0])
        else:
            author = '空'
    
    #publication_year
    raw_basic_info = soup.find_all('div', id = 'info')
    publication_year = re.findall(r"出版年:</span>(.+?)<br/>",str(raw_basic_info))
    if str(publication_year) != '[]':
        publication_year = publication_year[0][1:]
    else:
        publication_year = '空'
        
    #rating
    raw_basic_rating = soup.find_all('strong', class_ ="ll rating_num ")
    rating = dr.sub('',str(raw_basic_rating))
    if str(rating) != '[ ]':
        rating = float(rating[2:-2])
    else:
        rating = '评分人数不足'
    
    #rating_number
    raw_basic_rating_num = soup.find_all('span', property ="v:votes")
    rating_num = dr.sub('',str(raw_basic_rating_num))
    if str(rating_num) != '[]':
        rating_num = int(rating_num[1:-1])
    else:
        rating_num = '评分人数不足'

    #introduction
    raw_introduction = soup.find_all('div', class_ = 'intro')
    if str(raw_introduction) != '[]':
        introduction = dr.sub('',str(raw_introduction[0]))[1:]
    else:
        introduction = '空'
    
    #catalog
    raw_catalog = soup.find_all('div', {'class':'indent','style':'display:none'} )
    if raw_catalog != []:
        catalog = dr.sub('',str(raw_catalog[0]))[1:]
        catalog = re.sub('\n        ',' ',catalog)
        catalog = re.sub('\n',' ',catalog)
        catalog = re.sub(' ',' ',catalog)
        catalog = re.sub('·','',catalog)
    else:
        catalog = '暂无'
    
    #tag
    tag_list = []
    raw_tag = soup.find_all('a', class_ = 'tag')
    for i in range(len(raw_tag)):
        tag_list.append(dr.sub('',str(raw_tag[i])))

    book_info = list([title, author, publication_year, rating, rating_num, introduction, catalog, tag_list])
    
    return book_info

In [371]:
book_df = pd.DataFrame(columns=['title', 'author', 'publication_year', 'rating', 'rating_num', 'introduction', 'catalog', 'tag_list'])
n = len(url_list)
for i in range(n):
    book_info = get_book_info(url_list[i])
    book_df.loc[i] = book_info
    print(str(i) + '/' + str(n) + ' finished')
    print(pd.DataFrame(book_info))

/Users/wanghanxuan/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/wanghanxuan/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


0/980 finished
                                                   0
0                                            云彩收集者手册
1                                      [英] 加文·普雷特-平尼
2                                             2018-3
3                                                8.4
4                                               1596
5  天空的美，云知道。 赏云协会官方出版物，北京天文馆馆长朱进作序推荐。 ▼本书将帮您解锁以下新...
6          如何收集云彩 云的分类 十种主要云彩类型： 积云 层积云 层云 高积云 高层...
7                  [科普, 自然, 云, 气象, 博物, 生活, 英国, 2018]
1/980 finished
                                                   0
0                                           枪炮、病菌与钢铁
1                                                谢延光
2                                           2006-4-1
3                                                8.8
4                                              19370
5  为什么是欧亚大陆人征服、赶走或大批杀死印第安人、澳大利亚人和非洲人，而不是相反？为什么么小麦...
6          前言 耶利的问题 历史进程的地区差异 第一部分 从伊甸园到卡哈马卡 第一章 ...
7      [人类学, 历史, 社会学, 社会文化史, 杰拉德·戴蒙德, 社会, 科普, 生物地理学]
2/980 finished
 

17/980 finished
                                                   0
0                                              癌症·真相
1                                                 菠萝
2                                          2015-10-1
3                                                8.9
4                                               2217
5  荣获中宣部、中国图书评论学会和央视“2015中国好书”奖 荣获第十一届文津图书奖 荣获第八届...
6          癌症是什么 (1) 癌症和肿瘤 (2) 什么导致了癌症 (3) 癌症如何导致...
7                [科普, 癌症, 医学, 健康, 科学, 菠萝, 肿瘤君, 众病之王]
18/980 finished
                                                   0
0                                               超越时空
1                                                刘玉玺
2                                             2009-6
3                                                  9
4                                               3354
5  当代物理学中一些非常重要而又艰深的思想，往往因为难以形象浅显地解说而不易为公众了解。本书作者...
6          内容提要 作者简介 序言 致谢 第一篇　进入第五维 第一章　时空之外的世界 ...
7           [科普, 物理, 宇宙, 时空, 科学, 加来道雄, 物理学, 数学与自然科学]
19/980 finishe

34/980 finished
                                                   0
0                                           吃货的生物学修养
1                                           原点阅读系列丛书
2                                           2016-9-1
3                                                8.6
4                                               1110
5  食物空前丰富的工业社会，反而成了引发疾病的导火索。当人们真正了解了王立铭所著的《吃货的生物学...
6          第一章|脂肪的秘密 一连体的老鼠 二人类的群星闪耀时 三魔法王子和瘦素蛋白 ...
7          [科普, 生物学, 健康, 趣味科普, 医学健康, 王立铭, 食品营养, 科学史]
35/980 finished
                                                   0
0                                               病者生存
1                           [美]沙龙·莫勒姆（Sharon Moalem）
2                                          2018-7-20
3                                                9.1
4                                                184
5  日光浴能够降低胆固醇吗？ 为什么亚洲人喝酒容易脸红？ 为什么天冷的时候我们想撒尿？ 为什么地...
6          引言/1 第1章  走出补铁的误区/11 第2章  血糖里隐藏的秘密/38 ...
7             [科普, 医学, 健康, 生物, 医疗, 进化, 自然科学相关, 通俗读物]
36/980 finishe

51/980 finished
                                                   0
0                                               创造自然
1                                       [德] 安德烈娅·武尔夫
2                                            2017-10
3                                                8.9
4                                                954
5  每个在求知路上经历过憧憬与困顿交织的人，都可以从洪堡身上看到：一个人，可以如何用尽一生的心智...
6          前言 第一部　出发：新生的想法 1 开端 2 想象与自然：歌德与洪堡 3 寻...
7                [自然, 科普, 自然文学, 洪堡, 传记, 博物学, 历史, 博物]
52/980 finished
                                                   0
0                                      美国科学·生命科学·一年级
1                                      [美] 蒂莫西·库尼 等著
2                                             2017-3
3                                             评分人数不足
4                                             评分人数不足
5  读美国小学标准科学教材、学习科学知识、 掌握科学方法、形成科学思维、学会阅读科学 ........
6          第一章 生物与非生物 第二章 栖息地 第三章 植物与动物如何生活 第四章 生...
7             [亲子, 美国科学, 科普, 教材, 科普知识, 美国, 科学, 生命科学]
53/980 finishe

...

901/980 finished
                                                   0
0                                              决策与判断
1                                         [美]斯科特·普劳斯
2                                             2004-9
3                                                8.5
4                                               3000
5  本书的对象是希望了解决策与判断心理学的基础知识的非专业人士。它着重是实验结果而不是心理学理论...
6          读者调查 第一部分 知觉、记忆和情境 第一章 选择性知觉 第二章 认识不协调...
7            [心理学, 决策与判断, 思维, 决策, 社会心理学, 心理, 管理, 逻辑]
902/980 finished
                                                   0
0                                               时间简史
1                                             史蒂芬·霍金
2                                             2006-6
3                                                9.1
4                                                343
5  《时间简史·果壳中的宇宙》(典藏版)(套装共2册)收录了史蒂芬·霍金的《时间简史》插图版和《...
6                                                 暂无
7           [史蒂芬·霍金, 科学, 时间简史, 科普, 物理, 自然科学, 宇宙, 时间]
903/980 fini

918/980 finished
                                                   0
0                                           人类“吸猫”小史
1                                        [英] 艾比盖尔·塔克
2                                             2018-4
3                                                7.2
4                                                281
5  本书是一本破解人类为何吸猫成瘾，猫如何“统治”并“接管”世界的科普读物。作者艾比盖尔•塔克(...
6          前 言 001 第一章 地下墓穴 013 第二章 猫的发源地 032 第三章...
7               [猫, 科普, 宠物, 动物, 文化史, 社会学, 文化, 历史与记忆]
919/980 finished
                                                   0
0                                              惊人的假说
1                                        （英）弗朗西斯·克里克
2                                            2004-01
3                                                  8
4                                                525
5  40年前弗朗西斯·克里克与詹姆斯·沃森一道,发现DNA结构，改变了我们对生命本质的理解，从面...
6          原作者为中译本所作的序 前言 致谢 译校者序 第一部分 第一章 引言 第二章...
7             [科普, 脑科学, 意识, 心理学, 生物, 第一推动丛书, 科学, 哲学]
920/980 fini

935/980 finished
                                          0
0                                   法律常识全知道
1                                       春之霖
2                                 2010-10-1
3                                       7.9
4                                        36
5  法律常识全知道，ISBN：9787511306241，作者：春之霖，赵广娜 编著
6                                        暂无
7     [法律, 科普, 通识, 法学, 阅读-法律, 学习, 社会修炼, 专业]
936/980 finished
                                                   0
0                                     生活中的心理学-牛津通识读本
1                              [英] 吉莉恩•巴特勒 弗雷达•麦克马纳斯
2                                             2013-1
3                                                7.5
4                                                312
5  《牛津通识读本:生活中的心理学》介绍了心理学的各大主要学派，将心理学中的复杂概念（如知觉）做...
6          １　什么是心理学？怎样研究心理学？ ２　什么进入到我们的头脑里？知觉 ３　什...
7        [心理学, 牛津通识读本, 科普, 心理, 通识, 英国, 通识阅读, kindle]
937/980 finished
                                                   0
0                      

952/980 finished
                                                   0
0                                                意志力
1                                        [美] 罗伊·鲍迈斯特
2                                             2012-6
3                                                8.1
4                                               3002
5  工作低效？情感纠葛？减肥总是反弹？控制不住乱花钱？沉溺于社交媒体和在线娱乐？天天熬夜时间也不...
6          引 言 意志的衰退 /2 意志的回归 /6 进化和礼仪 为什么你要运用意志力...
7           [心理学, 意志力, 自我管理, 个人管理, 心理, 思维, 励志, 科学励志]
953/980 finished
                                                   0
0                                               亿亿万万
1                                           [美]卡尔·萨根
2                                            2018-10
3                                             评分人数不足
4                                             评分人数不足
5  这是卡尔•萨根在病床上完成的遗著，他用睿智的眼光审视了那些关乎21世纪人类命运的重大问题。这...
6          第一部分 量化之美与力量　001 第一章   亿亿万万　003 第二章  ...
7            [科普, 卡尔·萨根, 科学, 文化, 宇宙, 2018, 0.科学, 美国]
954/980 fini

970/980 finished
                                                   0
0                                              卑微的套套
1                                                 姜玢
2                                          2013-4-23
3                                                7.3
4                                                549
5  《卑微的套套:安全套进化史》作者安妮·科利尔采撷了人类历史上关于安全套这个小玩意儿的无数性感...
6          引言 第一章 纸莎草、大毒蛇和遮羞布： 古代人与安全套 第二章 从为了快乐的...
7                [趣味史, 冷知识, 科普, 历史, 文化, 闲趣, 美国, 文化史]
971/980 finished
                                                   0
0                                               肥志百科
1                                                 肥志
2                                             2018-9
3                                                8.6
4                                                 28
5  《肥志百科》是肥志继《历史是一群喵》系列后全新打造的漫画品牌，用漫画讲百科，用百科讲历史。涉...
6          1.给你一点正能量 地图炮的由来.........................
7                       [漫画, 科普, 肥志, 绘本, 有趣, 轻松, 中国]
972/980 fini

In [372]:
book_df.to_csv('Douban_popular_science_book_info.csv', index = False)

In [5]:
#read the data
data = pd.read_csv('Douban_popular_science_book_info.csv', lineterminator='\n')

In [6]:
data

,title,author,publication_year,rating,rating_num,introduction,catalog,tag_list
0,云彩收集者手册,[英] 加文·普雷特-平尼,2018-3,8.4,1596,天空的美，云知道。 赏云协会官方出版物，北京天文馆馆长朱进作序推荐。 ▼本书将帮您解锁以下新...,如何收集云彩 云的分类 十种主要云彩类型： 积云 层积云 层云 高积云 高层...,"['科普', '自然', '云', '气象', '博物', '生活', '英国', '2018']"
1,枪炮、病菌与钢铁,谢延光,2006-4-1,8.8,19370,为什么是欧亚大陆人征服、赶走或大批杀死印第安人、澳大利亚人和非洲人，而不是相反？为什么么小麦...,前言 耶利的问题 历史进程的地区差异 第一部分 从伊甸园到卡哈马卡 第一章 ...,"['人类学', '历史', '社会学', '社会文化史', '杰拉德·戴蒙德', '社会',..."
2,肠子的小心思,【德】朱莉娅·恩德斯,2016-1,9.0,2810,【内容简介】 《肠子的小心思》是德国图书界的黑马，口嫌体正直的德国人，一面对粑粑说着“不要不...,作者序 / 004 译者序 / 008 Part1 迷人的肠子/ 001...,"['科普', '健康', '医学', '健康类图书', '好书，值得一读', '脑洞很大',..."
3,哥德尔、艾舍尔、巴赫,严勇,1997-5,9.4,5269,集异璧－GEB，是数学家哥德尔、版画家艾舍尔、音乐家巴赫三个名字的前缀。《哥德尔、艾舍尔、巴...,目录: 作者为中文版所写的前言\r 译校者的话\r 概览\r 插图目示\r ...,"['科普', '哲学', '逻辑', '数学', '人工智能', '科学', '哥德尔', ..."
4,人类简史,[以] 尤瓦尔·赫拉利,2017-2-1,9.1,9809,【内容简介】 十万年前，地球上至少有六种不同的人， 但今日，世界舞台为什么只剩下我们自己？ ...,推荐序 高毅 第一部分 认知革命 1. 人类：一种也没什么特别的动物 2....,"['人类史', '历史', '独特视角', '社会', '科普', '哲学', '知识', ..."
5,哲学·科学·常识,陈嘉映,2018-3,8.7,853,【内容简介】 《哲学•科学•常识》是陈嘉映的代表作，这本书既是对历史上人类求知之路的回顾，也...,【导论】 科学认识 建构主义 哲学－科学 本书章节 ————上篇———— 【...,"['哲学', '科学哲学', '科普', '陈嘉映', '思维', '哲學', '科学史',..."
6,七堂极简物理课,[意] 卡洛·罗韦利,2016-5,8.3,6791,在本书中，意大利物理学家卡洛•罗韦利，用诗一般简洁优美的语言，向读者讲述了20世纪以来现代物...,目 录 自序 第一课 最美的理论 第二课 量子 第三课 宇宙的构造 ...,"['科普', '物理', '科学', '物理学', '自然科学相关', '涨知识啊，好书',..."
7,纸上动物园,[英] 夏洛特·斯莱,2017-12,8.5,500,世界文明的诺亚方舟，荡气回肠的自然史诗！ 《观察家》杂志年度书籍 《爱尔兰时报》年度非虚构...,引言 异域生物 本土生物 家养生物 怪异生物 注释 作者附言 拓展阅读 索引...,"['科普', '自然', '博物学', '科普博物', '英国', '艺术', '动物', ..."
8,地理学与生活,[美] 阿瑟·格蒂斯,2017-4,9.0,1347,地理学是一门古老的学科，最初仅指地球的绘图与勘查，但发展到今天已经逐渐成为一门范围广泛的学科...,前 言 第1章 绪 论 第2章 地 图 第一篇 地球科学传统 第3章 自然地...,"['地理学', '地理', '自然地理', '科普', '科普博物', '社会科学', '历..."
9,人类的明天,[法] 席里尔·迪翁,2018-8,8.4,402,★我们需要不焦虑的明天 面对不安全的食品、受污染的环境、无尽的消费重压、教育困境， 这本书提...,引言 1 出发 12 斯坦福大学：震颤背后 14 与伊丽莎白•海德利和安东尼...,"['科普', '未来', '环境', '好书，值得一读', '思维', '席里尔·迪翁', ..."
